In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
from langchain_chroma import Chroma
import json
import requests

In [2]:
load_dotenv()

True

In [3]:
headers = {
    "authorization": os.getenv("myserver_api_key")
}

embed_url = os.getenv("myserver_url")+":8000/embed"
rerank_url = os.getenv("myserver_url")+":8000/rerank"

In [4]:
class CustomEmbeddings:
    def embed_documents(self, texts):
        payload = {
            "texts": texts,
            "is_query": False
        }
        return requests.post(embed_url, json=payload, headers=headers).json()["embeddings"]
    
    def embed_query(self, text):
        payload = {
            "texts": [text],
            "is_query": True
        }
        return requests.post(embed_url, json=payload, headers=headers).json()["embeddings"][0]

In [5]:
embeddings = CustomEmbeddings()

In [6]:
vector_store = Chroma(
    persist_directory="../chroma_db",
    embedding_function=embeddings
)

In [7]:
# mmr 기반
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k":10, "fetch_k":50}
)

In [8]:
query = "휴가"

In [11]:
docs = retriever.invoke(query)

In [12]:
docs

[Document(id='ad59eaa3-ec83-45d3-a9c6-c5edc5fca186', metadata={}, page_content='법정 휴가 규정'),
 Document(id='b4cefeb4-5de7-4b1c-af69-0288b7df75bf', metadata={}, page_content='CoreFlow는 근로자의 충분한 휴식을 보장하고 일과 삶의 균형을 지원하기 위해 근로기준법 및 관련 법규에 따른 휴가 제도를 운영한다'),
 Document(id='5108c9d4-89f2-4309-9328-8ec6793e2426', metadata={}, page_content='이는 부모, 배우자, 자녀 등 돌봄이 필요한 가족을 위해 사용하는 휴가로, 연간 최대 10일까지 하루 단위로 분할하여 사용할 수 있습니다'),
 Document(id='fbae571c-f7a1-4b74-b54c-54ea63f35ec2', metadata={}, page_content='정의 및 적용 범위: 재택근무는 부여받은 업무를 자택 등 지정된 장소에서 수행하는 근무 형태를 의미한다'),
 Document(id='2fd80cc1-91cf-41f2-a90c-c89bc3ce8956', metadata={}, page_content='사내 복지 시설: 사내 휴게실, 식당 등을 운영하여 편의를 제공하고, 필요에 따라 휴양 시설 등을 임차하여 직원들이 이용할 수 있도록 지원한다'),
 Document(id='8d3bc264-facb-4c6f-a09e-e427c4f1e1ce', metadata={}, page_content='5배, 휴일근로는 8시간 이내는 1'),
 Document(id='e463751c-39b8-4767-89b4-8aeadc68d527', metadata={}, page_content='조사 기간 중 피해자가 요청하는 경우, 근무 장소 변경, 유급 휴가 명령 등 피해자를 보호하기 위한 적절한 조치를 취할 수 있다'),
 Document(id='ab71e4fe-c9d9

In [ ]:
for doc in docs:
    print(doc.page_content)
    break

In [ ]:
parent_docs_df = pd.read_csv("../data/parent_split_docs_20250909.csv")
parent_docs_list = parent_docs_df["docs"].tolist()
len(parent_docs_list)

In [ ]:
parent_docs_list[:5]

In [ ]:
def search_parent_docs(docs, parent_docs):
    searched_docs = []
    for doc in docs:
        doc_content = doc.page_content
        for parent_doc in parent_docs:
            if doc_content in parent_doc:
                searched_docs.append(parent_doc)
    
    return searched_docs

In [ ]:
searched_parent_docs = search_parent_docs(docs, parent_docs_list)
searched_parent_docs_df = pd.DataFrame({"docs":searched_parent_docs})
print(searched_parent_docs_df.shape)
searched_parent_docs_df = searched_parent_docs_df.drop_duplicates()
searched_parent_docs = searched_parent_docs_df["docs"].tolist()
len(searched_parent_docs)

In [ ]:
searched_parent_docs

In [ ]:
response = requests.post(rerank_url, json={"query":query, "documents":searched_parent_docs}, headers=headers)
response

In [ ]:
response.json()

In [ ]:
best_results = []
for result in response.json()["results"][:5]:
    best_results.append(result["text"])
print(len(best_results))

In [ ]:
def chat_with_ollama(message):
    model=os.getenv("ollama_model")
    url = os.getenv("ollama_url") + "/chat"
    
    messages = []
    messages.append({"role": "system", "content": """
너의 이름은 CoreFlow AI 이고, 너의 역할은 회사 내부 규정에 대한 질문에 답변해 주는 거야. 사용자 프롬프트 포맷은 아래와 같아.

# 질문:
질문 내용
# 첨부된 문서:
문서 내용

첨부된 문서를 활용해서 최대한 자세하게 대답해주되, 모르겠으면 솔직히 모르겠다고 대답해줘.
"""})
    messages.append({"role": "user", "content": f"""
# 질문:
{message}
# 첨부된 문서:
{best_results}
"""})
    # messages.append({"role": "user", "content": message})
    
    payload = {
        "model": model,
        "messages": messages,
        "stream": False,
        "options": {
            "num_ctx": 128000
        }
    }
    
    response = requests.post(url, json=payload).json()["message"]["content"]
    messages.append({"role": "assistant", "content": response})
    return response

In [ ]:
model_response = chat_with_ollama(query)
print(model_response)